In [5]:
import requests, os
from urllib import parse

# 使用网络爬虫获取所需要训练的图片

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
img_url = 'https://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&ct=201326592&is=&fp=result&queryWord={0}&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=-1&z=&ic=&hd=&latest=&copyright=&word={0}&s=&se=&tab=&width=&height=&face=0&istype=2&qc=&nc=1&fr=&expermode=&force=&cg=star&pn={1}&rn=30&gsm=&1578807000105='

def getIdolPicture(keyword, dest_dir, batch):
    # 建立图片储存路径
    if not os.path.exists(dest_dir):
        os.mkdir(dest_dir)
        
    for i in range(batch):
        res = requests.get(img_url.format(parse.quote(keyword), i*30))
        res.encoding = 'utf-8'
        print(len(res.json()['data']))
        for ele in res.json()['data']:
            url = ele.get('thumbURL')
            if url:
                # 获取图片链接最后部分并做为图片名字
                with open(dest_dir + url.split('/')[-1], 'wb') as f:
                    res2 = requests.get(url, headers=headers)
                    f.write(res2.content)


In [9]:
getIdolPicture('范冰冰', 'idol1/', 10)

31
31
31
31
31
31
31
31
31
31


In [11]:
getIdolPicture('迪丽热巴', 'idol2/', 10)

31
31
31
31
31
31
31
31
31
31


In [12]:
getIdolPicture('林志玲', 'idol3/', 10)

31
31
31
31
31
31
31
31
31
31


# 使用OpenCV 撷取人脸

In [82]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from PIL import Image

In [83]:
face_cascade = cv.CascadeClassifier('/usr/local/Cellar/opencv/4.2.0_1/share/opencv4/haarcascades/haarcascade_frontalface_default.xml')

def extractFace(filename, src_dir, dest_dir):
    img = cv.imread(src_dir + filename)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    im = Image.open(src_dir + filename)
    
    for (x, y, w, h) in faces:
        roi_color = img[y:y+h, x:x+w]
        box = (x, y, x+w, y+h)
        crpim = im.crop(box).resize((64, 64))
        crpim.save(dest_dir + filename)
        

In [24]:
if not os.path.exists('idol1_face'):
    os.mkdir('idol1_face')

for f in os.listdir('idol1/'):
    try:
        extractFace(f, 'idol1/', 'idol1_face/')
    except:
        print(f)

u=4252810034,3548750092&fm=11&gp=0.jpg


In [32]:
if not os.path.exists('idol2_face'):
    os.mkdir('idol2_face')

for f in os.listdir('idol2/'):
    try:
        extractFace(f, 'idol2/', 'idol2_face/')
    except:
        print(f)

u=2131554868,2623628550&fm=26&gp=0.jpg


In [34]:
if not os.path.exists('idol3_face'):
    os.mkdir('idol3_face')

for f in os.listdir('idol3'):
    try:
        extractFace(f, 'idol3/', 'idol3_face/')
    except:
        print(f)

u=1980474683,3378689882&fm=26&gp=0.jpg


# 利用卷积神经网络识别明星图片

## 建构卷积神经网络

In [61]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [62]:
# Initialising the CNN
classifier = Sequential()

# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Convolution
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
classifier.add(Flatten())

# Fully Connected
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

classifier.compile(optimizer = 'adam',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

## 图像增广技术

In [63]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                  shear_range = 0.2,    # x 坐标保持不变， 而对应y 的坐标按比例发生平移
                                  zoom_range = 0.2,     # 可以让图片在长或宽的方向进行放大
                                  horizontal_flip = True  # 水平翻转操作
                                  )

In [64]:
test_datagen = ImageDataGenerator(rescale = 1. / 255)

## 建立训练与测试数据集

In [65]:
training_set = train_datagen.flow_from_directory(
    'trainset/', target_size=(64, 64),
    batch_size=30, 
    class_mode='categorical'
)

Found 490 images belonging to 3 classes.


In [66]:
test_set = test_datagen.flow_from_directory(
    'testset/', target_size=(64, 64),
    batch_size=30,
    class_mode='categorical'
)

Found 71 images belonging to 3 classes.


## 训练神经网络

In [67]:
history = classifier.fit_generator(
    training_set, 
    epochs = 100,
    verbose=1,
    validation_data= test_set
)

Epoch 1/100
17/17 [==============================] - 1s 87ms/step - loss: 1.1266 - accuracy: 0.3796 - val_loss: 1.0812 - val_accuracy: 0.3380
Epoch 2/100
17/17 [==============================] - 1s 82ms/step - loss: 1.0720 - accuracy: 0.4163 - val_loss: 1.0172 - val_accuracy: 0.4085
Epoch 3/100
17/17 [==============================] - 1s 78ms/step - loss: 1.0136 - accuracy: 0.4878 - val_loss: 0.8654 - val_accuracy: 0.5634
Epoch 4/100
17/17 [==============================] - 1s 83ms/step - loss: 0.9834 - accuracy: 0.5286 - val_loss: 0.8895 - val_accuracy: 0.6479
Epoch 5/100
17/17 [==============================] - 1s 78ms/step - loss: 0.9135 - accuracy: 0.5878 - val_loss: 0.8661 - val_accuracy: 0.5775
Epoch 6/100
17/17 [==============================] - 1s 80ms/step - loss: 0.8971 - accuracy: 0.5796 - val_loss: 0.8937 - val_accuracy: 0.6338
Epoch 7/100
17/17 [==============================] - 1s 81ms/step - loss: 0.8039 - accuracy: 0.6367 - val_loss: 0.8423 - val_accuracy: 0.6620
Epoch 

# 预测单张图片

In [222]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from PIL import Image

## 截取图片的面部特征
face_cascade = cv.CascadeClassifier('/usr/local/Cellar/opencv/4.2.0_1/share/opencv4/haarcascades/haarcascade_frontalface_default.xml')
img = cv.imread('timg.jpeg')
faces = face_cascade.detectMultiScale(img, 1.3, 5)
im = Image.open('timg.jpeg')

for (x, y, w, h) in faces:
    roi_color = img[y:y+h, x:x+w]
    box = (x, y, x+w, y+h)
    crpim = im.crop(box).resize((64, 64))
    crpim.save('test3.jpeg')
    

In [176]:
from keras.preprocessing import image

test_image = image.load_img('prediction_dataset/i1_d.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0)
classifier.predict_classes(test_image)

array([0])

In [169]:
test_image.shape

(1, 64, 64, 3)

expand_dims(a, axis)就是在axis的那一个轴上把数据加上去，这个数据在axis这个轴的0位置。

例如原本为一维的2个数据，axis=0，则shape变为(1,2),axis=1则shape变为(2,1)

再例如 原本为 (2,3),axis=0，则shape变为(1,2,3),axis=1则shape变为(2,1,3)

In [218]:
for f in os.listdir('prediction_dataset/'):
    if f != '.DS_Store':
        test_image = image.load_img('prediction_dataset/{}'.format(f), target_size=(64, 64))
        test_image = image.img_to_array(test_image)

        test_image = np.expand_dims(test_image, axis = 0)
        num = classifier.predict_classes(test_image)
        if num == 0:
            print('图片：' + f + ' 是范冰冰')
        elif num == 1:
            print('图片：' + f + ' 是迪丽热巴')
        elif num == 2:
            print('图片：' + f + ' 是林志玲')

图片：范冰冰3.jpg 是范冰冰
图片：范冰冰2.jpg 是范冰冰
图片：林志玲4.jpeg 是林志玲
图片：林志玲3.jpg 是林志玲
图片：林志玲2.jpg 是林志玲
图片：林志玲.jpg 是林志玲
图片：迪丽热巴3.jpeg 是迪丽热巴
图片：迪丽热巴.jpeg 是迪丽热巴
图片：范冰冰.jpg 是范冰冰
图片：迪丽热巴2.jpg 是林志玲
图片：迪丽热巴3.jpg 是迪丽热巴
